In [9]:
import numpy as np
import tensorflow as tf

In [1]:
file_path = r'D:\CodeHub\MyData\cmn.txt'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
except FileNotFoundError:
    print(f"错误：文件 {file_path} 未找到。")
except Exception as e:
    print(f"读取文件时发生错误：{e}")


In [5]:
pairs = [line.split('\t') for line in text.splitlines()]
np.random.shuffle(pairs)
sentences_en, sentences_cmn = zip(*(pair[:2] for pair in pairs))



In [8]:
for i in range(5):
    print(f"{sentences_en[i]} => {sentences_cmn[i]}")


Life cannot exist without water. => 水是生命的源泉。
You'd do that if Tom asked you to, wouldn't you? => 若汤姆请求你的话，你是会做的，不是吗？
My father told me where to go. => 我爸爸告诉我去哪儿。
The conference is to be held in Tokyo the day after tomorrow. => 会议将在后天于东京举行。
Birds fly. => 鳥類飛行。


In [17]:
vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_sequence_length=max_length,
)
text_vec_layer_cmn = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_sequence_length=max_length,
)

text_vec_layer_en.adapt(sentences_en)
text_vec_layer_cmn.adapt([f"startofseq {s} endofseq" for s in sentences_cmn])


In [18]:
text_vec_layer_en.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('the'),
 np.str_('i'),
 np.str_('to'),
 np.str_('you'),
 np.str_('a'),
 np.str_('is'),
 np.str_('tom'),
 np.str_('he')]

In [19]:
text_vec_layer_cmn.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('startofseq'),
 np.str_('endofseq'),
 np.str_('多少錢？'),
 np.str_('请稍等一下。'),
 np.str_('每个人都知道汤姆的法语很好。'),
 np.str_('我有点累了。'),
 np.str_('聖誕節快到了。'),
 np.str_('我也是這麼想的。')]

In [ ]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_cmn[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_cmn[100_000:]])
Y_train = text_vec_layer_cmn([f"endofseq {s}" for s in sentences_cmn[:100_000]])
Y_valid = text_vec_layer_cmn([f"endofseq {s}" for s in sentences_cmn[100_000:]])

In [21]:
encoder_input = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_input = tf.keras.layers.Input(shape=[], dtype=tf.string)

embed_size = 128
encoder_input_ids = text_vec_layer_cmn(encoder_input)
decoder_input_ids = text_vec_layer_en(decoder_input)
encoder_embeddings = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)(encoder_input_ids)
decoder_embeddings = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)(decoder_input_ids)

encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

decoder = tf.keras.layers.LSTM(512, return_state=True)
decoder_outputs, *decoder_state = decoder(decoder_embeddings, initial_state=encoder_state)

output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

In [24]:
model = tf.keras.Model(inputs=[encoder_input, decoder_input], outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10, validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 50), output.shape=(None, 1000)